In [1]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import fetch_california_housing as fch #加利福尼亚房屋价值数据集
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# 不发出警告
import warnings
warnings.filterwarnings('ignore') 
# statistics
from statsmodels.distributions.empirical_distribution import ECDF

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import os
from datetime import datetime
from time import time
#plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
#plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
#Scikit-Learn for Modeling
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split

import missingno as msno
print('已导入必要第三方库')

已导入必要第三方库


In [2]:
import re
import jieba # 结巴分词
# gensim用来加载预训练word vector
from gensim.models import KeyedVectors
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
import jieba.analyse
import jieba.posseg as psg

In [3]:
train=pd.read_csv('train.csv')

In [4]:
train.head()

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


In [5]:
train['subject'].unique()

array(['价格', '配置', '操控', '舒适性', '油耗', '动力', '内饰', '安全性', '空间', '外观'],
      dtype=object)

In [6]:
train_gby=train.groupby(['subject','sentiment_value'])['content_id'].count().reset_index()

In [7]:
test=pd.read_csv('test_public.csv')

In [8]:
test.head(2)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊


In [9]:
subject_list={'价格':1, '配置':2, '操控':3
              ,'舒适性':4, '油耗':5,'动力':6, '内饰':7, '安全性':8, '空间':9, '外观':10}

In [10]:
train['subject_n']=train['subject'].map(subject_list)

In [11]:
train.head(3)

,content_id,content,subject,sentiment_value,sentiment_word,subject_n
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响,1
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高,1
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低,1


In [12]:
#对content 内容进行词向量处理
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cuttxt=lambda x:" ".join(jieba.lcut(x))
train['cleantxt']=train['content'].apply(cuttxt)
test['cleantxt']=test['content'].apply(cuttxt)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/wt/xn04w94s0kldh3whfsyvnsph0000gn/T/jieba.cache
Loading model cost 0.763 seconds.
Prefix dict has been built successfully.


In [14]:
countvec=CountVectorizer()
wordmtx=countvec.fit_transform(train['cleantxt'])

In [15]:
tf_model=countvec.fit(train['cleantxt'])

In [16]:
x_train=tf_model.transform(train['cleantxt'])
x_test=tf_model.transform(test['cleantxt'])
y_train=train['sentiment_value']


In [17]:
#用贝叶斯方法预测文本的情绪
from sklearn import naive_bayes

In [18]:
NBmodel=naive_bayes.MultinomialNB()

NBmodel.fit(x_train,y_train)

MultinomialNB()

In [19]:
y_pred=NBmodel.predict(x_test)

In [45]:
#情绪的预测结果为pre_list
pre_list=list(y_pred)

AttributeError: 'list' object has no attribute 'map'

In [21]:
#用随机森林进行subject的预测
from sklearn.tree import DecisionTreeClassifier

In [22]:
x_subject=train['subject_n']

In [23]:
clf=DecisionTreeClassifier()

In [24]:
clf.fit(x_train,x_subject)

DecisionTreeClassifier()

In [25]:
y_subject=clf.predict(x_test)

In [26]:
#subject_list为内容的subject预测结果
subject_list=list(y_subject)

In [27]:
len(train)

9947

In [36]:
test.head()

,content_id,content,cleantxt
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！,欧蓝德 ， 价格便宜 ， 森林 人 太贵 啦 ！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊,楼主 什么 时候 提 的 车 ， 南昌 优惠 多少 啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜,吉林 ， 2.5 优惠 20000 ， 送 三年 九次 保养 ， 贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。,便宜 2 万 的 豪华 特装 ， 实用 配制 提升 ， 优惠 还给 力 ， 确实 划算 。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪,如果 实在 想 买 就 等 车展 期间 ， 优惠 2 万 ， 我 24.98 万入 的 2.5 豪


In [42]:
sub=pd.read_csv('submit_example.csv')

In [43]:
sub.head()

,content_id,subject,sentiment_value,sentiment_word
0,XuPwKCnA2fqNh5vm,价格,0,NaN
1,2jNbDn85goX3IuPE,价格,0,NaN
2,hLgEADQ8sUnvGFK9,价格,0,NaN
3,nZmM7LQsfr03wUaz,价格,0,NaN
4,pwd8MnrthDqLZafe,价格,0,NaN


In [44]:

sub['content_id']=test['content_id']
sub['subject']=subject_list
sub['sentiment_value']=pre_list

print(sub.head())

         content_id  subject  sentiment_value  sentiment_word
0  XuPwKCnA2fqNh5vm        1                1             NaN
1  2jNbDn85goX3IuPE        1                0             NaN
2  hLgEADQ8sUnvGFK9        1                0             NaN
3  nZmM7LQsfr03wUaz        1                1             NaN
4  pwd8MnrthDqLZafe        1                0             NaN


In [47]:
subject_list_t={1:'价格',2:'配置', 3:'操控',4:'舒适性',5:'油耗',6:'动力',7:'内饰',8:'安全性',9:'空间',10:'外观'}

In [48]:
sub['subject']=sub['subject'].map(subject_list_t)

In [50]:
sub.to_csv('sub_1st.csv')